In [1]:
import pandas as pd
import csv
import datetime
import yfinance as yf
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup as bs
import requests
from pyspark.sql.functions import sum,max,min,mean,count
import datetime as dt
import pyspark
from pyspark.sql import SparkSession
import findspark
import yaml
from yaml.loader import SafeLoader
from os.path import abspath

warehouse_location = abspath('spark-warehouse')
with open('cfg.yml') as f:
    config = yaml.load(f, Loader = SafeLoader)

    #create spark connection
findspark.init()
spark = SparkSession.builder \
    .master(config['spark']['spark_master'])\
    .appName('retrieve')\
    .enableHiveSupport()\
    .config('spark.sql.warehouse.dir', warehouse_location)\
    .config(config['spark']['spark_jars'], config['spark']['spark_jars_path'])\
    .config('spark.cores.max', '2')\
    .config('spark.executor.cores', '2')\
    .getOrCreate()
spark.sparkContext.setLogLevel("WARN")
spark

#create database config details
url = config['postgres']['url']
properties = {
    'user': config['postgres']['user'],
    'password' : config['postgres']['user'],
    'url': url,
    'driver': config['postgres']['driver']
    
}

23/03/28 17:57:29 WARN Utils: Your hostname, cis6180 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/03/28 17:57:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/03/28 17:57:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Retrieve Data from Database and Write to Csv

In [3]:
import pyspark.pandas as ps

#retrieve data from database
def return_data(ticker_list, from_date, to_date):
    sentiment = spark.read.format("jdbc")\
        .option("url", "jdbc:postgresql://localhost:5432/financials") \
        .option("driver", "org.postgresql.Driver").option("dbtable", "sentiment") \
        .option("user", "adam").option("password", "green").load()
    finance = spark.read.format("jdbc")\
        .option("url", "jdbc:postgresql://localhost:5432/financials") \
        .option("driver", "org.postgresql.Driver").option("dbtable", "company_data") \
        .option("user", "adam").option("password", "green").load()
    full_data = finance.join(sentiment, ['date', 'ticker'], 'left').fillna(0)
    full_data = full_data.toPandas()
    df_list = []
    for ticker in ticker_list:
        working_data = full_data[full_data['ticker'] == ticker]
        print(ticker, working_data.shape)
        working_data = working_data.set_index('date').sort_values(by = 'date', ascending = True).loc[from_date:to_date,]
        print(ticker,  working_data.shape)
        working_data = working_data[~working_data.index.duplicated()]
        print(ticker, working_data.shape)
        working_data.to_csv('data/'+ticker+'_dataframe.csv')
        df_list.append(working_data)
return_data(['MSFT', 'GOOG'], "2016-01-01", "2023-03-01")

MSFT (6966, 21)
MSFT (6925, 20)
MSFT (1802, 20)
GOOG (3678, 21)
GOOG (3641, 20)
GOOG (1802, 20)


In [ ]:
spark.stop()